6/14/20 This notebook worked with stack version 2020_11

In [22]:
import os
import numpy as np
import pandas as pd

from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
from lsst.daf.persistence import Butler

from lsst.sims.utils import angularSeparation

from ast import literal_eval

### Load the data needed

In [23]:
dd = pd.read_csv('raftInstall.csv',index_col=0)

In [24]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" 
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

/home/bxin/lsst_stack/obs_lsst/lsstcam/CALIB


### Prepare vignetting function

In [25]:
# below we use v3.11 values
vfile = f"{os.environ['HOME']}/notebooks/f_factors/data/vignettingF.txt"
M1D = 8.36 #clear aperture as in Optical design
aa = np.loadtxt(vfile, skiprows=12)
vr = aa[:,0]
vv = aa[:,1]

### Create the dataframe

In [26]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']
properties = ['vendor', 'CCD', 'EO_run','ampX(pixel)', 'ampY(pixel)', 'ampX(deg)', 'ampY(deg)', 'fieldR(deg)', 'gain', 'readnoise', 
              'vignetting', 'effarea(m^2)']+ ['m5_%s'%i for i in filters] + ['fS'] \
     + ['T%s'%i for i in filters] + ['S%s'%i for i in filters] 
amps = ()
for det in cam:
    rname, dname = det.getName().split('_')
    if len(rname)>3:
        continue
    if np.isnan(dd.run[rname]):
        print('no data yet, use default', rname, dname)
        continue
    for i in range(8):
        amps = amps + ('%s_%s_C0%d'%(rname,dname,i),)
    for i in range(8):
        amps = amps + ('%s_%s_C1%d'%(rname,dname,i),)
mdf = pd.DataFrame(index=amps, columns=properties, dtype=object)

no data yet, use default R00 SG0
no data yet, use default R00 SG1
no data yet, use default R00 SW0
no data yet, use default R00 SW1
no data yet, use default R04 SG0
no data yet, use default R04 SG1
no data yet, use default R04 SW0
no data yet, use default R04 SW1
no data yet, use default R40 SG0
no data yet, use default R40 SG1
no data yet, use default R40 SW0
no data yet, use default R40 SW1
no data yet, use default R44 SG0
no data yet, use default R44 SG1
no data yet, use default R44 SW0
no data yet, use default R44 SW1


### Fill the dataframe

In [27]:
for det in cam:
    rname, dname = det.getName().split('_')
    if len(rname)>3:
        continue
    if np.isnan(dd.run[rname]):
        print('no data yet, use default', rname, dname)
        continue
    vendor = det.getSerial()[:3]
    ccdname = det.getSerial()
    for iamp, amp in enumerate(det):
        i = amp.getName()
        idx = '%s_%s_%s'%(rname,dname,i)
        mdf.loc[idx, 'vendor'] = vendor
        mdf['CCD'].loc[idx]  = ccdname
        mdf['EO_run'][idx]  = int(dd.run[rname])
        amp_point = amp.getBBox().getCenter()
        mdf['ampX(pixel)'].loc[idx] = amp_point.x
        mdf['ampY(pixel)'].loc[idx] = amp_point.y
        raDec = det.transform(amp_point, PIXELS, FIELD_ANGLE) 
        [mdf['ampX(deg)'].loc[idx], mdf['ampY(deg)'].loc[idx]] = np.round(np.degrees(raDec),4)
        radius = angularSeparation(0., 0., mdf['ampX(deg)'][idx], mdf['ampY(deg)'][idx])
        mdf['fieldR(deg)'].loc[idx] = np.round(radius,4)
        mdf['gain'].loc[idx] = np.round(amp.getGain(),4)
        mdf['readnoise'].loc[idx] = np.round(amp.getReadNoise(),4)
        mdf['vignetting'].loc[idx] = np.round(np.interp(radius, vr, vv),4)
        mdf['effarea(m^2)'].loc[idx] = np.round(mdf['vignetting'][idx]*np.pi*(M1D/2)**2,3)
        
        #print(iamp, i)
        if iamp>7:
            ii = 15-iamp
        else:
            ii = iamp+8
        dfDir = os.path.join('m5_output', rname)
        
        #test we got the order right:
        #dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
        #df = pd.read_csv(dfPath, index_col=0)     
        #print(mdf['readnoise'].loc[idx], df[idx[:7]].apply(literal_eval)['readnoise'][ii] )
        
        dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
        df = pd.read_csv(dfPath, index_col=0)
        for f in filters:
            mdf['m5_%s'%f].loc[idx] = np.round(df[idx[:7]].apply(literal_eval)[f][ii],3)
        mdf['fS'].loc[idx] = np.round(df[idx[:7]].apply(literal_eval)['fS'][ii],3)
            
        dfPath = os.path.join(dfDir, 'Tdf_%s.csv'%rname)
        df = pd.read_csv(dfPath, index_col=0)
        for f in filters:
            mdf['T%s'%f].loc[idx] = np.round(df[idx[:7]].apply(literal_eval)[f][ii],3)
            
        dfPath = os.path.join(dfDir, 'Sdf_%s.csv'%rname)
        df = pd.read_csv(dfPath, index_col=0)
        for f in filters:
            mdf['S%s'%f].loc[idx] = np.round(df[idx[:7]].apply(literal_eval)[f][ii],3)
#    break

no data yet, use default R00 SG0
no data yet, use default R00 SG1
no data yet, use default R00 SW0
no data yet, use default R00 SW1
no data yet, use default R04 SG0
no data yet, use default R04 SG1
no data yet, use default R04 SW0
no data yet, use default R04 SW1
no data yet, use default R40 SG0
no data yet, use default R40 SG1
no data yet, use default R40 SW0
no data yet, use default R40 SW1
no data yet, use default R44 SG0
no data yet, use default R44 SG1
no data yet, use default R44 SW0
no data yet, use default R44 SW1


In [28]:
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  
mdf[1600:1600+16]

,vendor,CCD,EO_run,ampX(pixel),ampY(pixel),ampX(deg),ampY(deg),fieldR(deg),gain,readnoise,vignetting,effarea(m^2),m5_u,m5_g,m5_r,m5_i,m5_z,m5_y,fS,Tu,Tg,Tr,Ti,Tz,Ty,Su,Sg,Sr,Si,Sz,Sy
R23_S01_C00,E2V,E2V-CCD250-239,11852,255.5,1000.5,0.6061,-0.2924,0.6729,1.1662,5.2779,0.6204,34.054,24.296,24.991,24.502,24.027,23.422,22.461,1.246,0.042,0.135,0.114,0.081,0.054,0.024,0.067,0.164,0.128,0.089,0.058,0.028
R23_S01_C01,E2V,E2V-CCD250-239,11852,767.5,1000.5,0.6345,-0.2924,0.6986,1.17,5.3742,0.6199,34.027,24.293,24.989,24.501,24.026,23.421,22.46,1.244,0.042,0.135,0.114,0.081,0.054,0.024,0.068,0.165,0.128,0.089,0.058,0.028
R23_S01_C02,E2V,E2V-CCD250-239,11852,1279.5,1000.5,0.6629,-0.2924,0.7245,1.1793,5.3333,0.6202,34.043,24.298,24.99,24.501,24.026,23.421,22.46,1.245,0.043,0.135,0.114,0.081,0.054,0.024,0.068,0.165,0.128,0.089,0.058,0.028
R23_S01_C03,E2V,E2V-CCD250-239,11852,1791.5,1000.5,0.6914,-0.2924,0.7507,1.1787,5.317,0.6198,34.022,24.3,24.99,24.501,24.026,23.42,22.459,1.245,0.043,0.135,0.114,0.081,0.054,0.024,0.068,0.165,0.128,0.089,0.058,0.028
R23_S01_C04,E2V,E2V-CCD250-239,11852,2303.5,1000.5,0.7198,-0.2924,0.7769,1.1909,5.4045,0.6199,34.027,24.298,24.99,24.501,24.026,23.421,22.459,1.245,0.043,0.136,0.114,0.081,0.054,0.024,0.069,0.165,0.128,0.089,0.058,0.028
R23_S01_C05,E2V,E2V-CCD250-239,11852,2815.5,1000.5,0.7483,-0.2924,0.8034,1.1866,5.3916,0.6194,34,24.304,24.993,24.503,24.028,23.423,22.461,1.251,0.043,0.136,0.114,0.082,0.055,0.024,0.069,0.166,0.128,0.089,0.058,0.028
R23_S01_C06,E2V,E2V-CCD250-239,11852,3327.5,1000.5,0.7767,-0.2924,0.8299,1.1867,5.3358,0.6189,33.972,24.304,24.99,24.5,24.024,23.42,22.458,1.244,0.043,0.135,0.113,0.081,0.054,0.024,0.069,0.165,0.127,0.089,0.058,0.028
R23_S01_C07,E2V,E2V-CCD250-239,11852,3839.5,1000.5,0.8052,-0.2924,0.8566,1.1945,5.4293,0.6185,33.95,24.299,24.988,24.499,24.024,23.42,22.459,1.242,0.043,0.136,0.114,0.081,0.054,0.024,0.069,0.165,0.128,0.089,0.058,0.028
R23_S01_C10,E2V,E2V-CCD250-239,11852,255.5,3002.5,0.6061,-0.1812,0.6326,1.1817,5.4649,0.621,34.087,24.306,24.988,24.498,24.026,23.425,22.463,1.25,0.043,0.135,0.113,0.081,0.055,0.024,0.07,0.164,0.127,0.089,0.058,0.028
R23_S01_C11,E2V,E2V-CCD250-239,11852,767.5,3002.5,0.6345,-0.1812,0.6599,1.1761,5.4381,0.6207,34.071,24.309,24.99,24.5,24.028,23.426,22.464,1.254,0.044,0.136,0.113,0.082,0.055,0.024,0.07,0.165,0.127,0.089,0.058,0.028


In [29]:
189*16

3024

In [30]:
len(amps)

3024

In [31]:
dfPath = os.path.join('m5_output', 'm5_master_db.csv')
mdf.to_csv(dfPath)

In [32]:
dfPath

'm5_output/m5_master_db.csv'

In [33]:
for f in filters:
    idx = mdf['m5_%s'%f]>0
    a = sum(idx)
    b = sum(mdf['m5_%s'%f]>-100)
    c = np.median(mdf['m5_%s'%f])
    d = np.median(mdf.loc[idx, 'm5_%s'%f])
    m1 = np.mean(mdf['m5_%s'%f])
    m2 = np.mean(mdf.loc[idx, 'm5_%s'%f])
    print('%s, %.0f/%.0f = %.3f, medianAll = %.1f, medianLive = %.1f, MeanAll = %.1f, meanLive = %.1f'%(f, a,b,a/b, c, d, m1, m2))

u, 3021/3024 = 0.999, medianAll = 24.2, medianLive = 24.2, MeanAll = 24.1, meanLive = 24.1
g, 3021/3024 = 0.999, medianAll = 25.0, medianLive = 25.0, MeanAll = 24.9, meanLive = 24.9
r, 3021/3024 = 0.999, medianAll = 24.5, medianLive = 24.5, MeanAll = 24.4, meanLive = 24.4
i, 3021/3024 = 0.999, medianAll = 24.0, medianLive = 24.0, MeanAll = 24.0, meanLive = 24.0
z, 3021/3024 = 0.999, medianAll = 23.4, medianLive = 23.4, MeanAll = 23.4, meanLive = 23.4
y, 3021/3024 = 0.999, medianAll = 22.5, medianLive = 22.5, MeanAll = 22.4, meanLive = 22.5


In [34]:
mdf['vignetting'].max()

0.6225

In [37]:
mdf['m5_u']['R01_S01_C17']

-1.0

In [40]:
mdf['Tu']['R01_S01_C17']

-1.0

In [41]:
mdf['Su']['R01_S01_C17']

-1.0